In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os
# 로컬 이미지 읽어 처리하는데 필요한 라이브러리
from io import BytesIO
import base64
from PIL import Image

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [2]:
def encode_image(image):
    buffered = BytesIO()  # 메모리에 임시로 데이터를 저장할 공간을 만든다
    w, h = image.size      # 이미지의 가로(w), 세로(h) 크기를 구함

    # 너무 큰 이미지는 축소해서 512x512 이하로 만듦
    if w > 512 or h > 512:
        scale = 512 / max(w, h)
    else:
        scale = 1.0

    resize_im = image.resize((int(w * scale), int(h * scale)))  # 비율 유지한 채로 크기 조정
    resize_im.save(buffered, format="JPEG")  # JPEG 형식으로 메모리에 저장

    # base64 문자열로 인코딩해서 반환 (GPT API는 base64 문자열로 이미지 받음)
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str

In [3]:

def img_TextGeneration(prompt, image1, image2):
    client = OpenAI(api_key = api_key)
    response = client.chat.completions.create(
        model = "gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type":"text",
                        "text": prompt
                    },
                    {
                        "type":"image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64, {encode_image(image1)}"
                        }
                    },
                     {
                        "type":"image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64, {encode_image(image2)}"
                        }
                    }
                ]
            }
        ]        
    )
    return response.choices[0].message.content


In [4]:
prompt1 = f"""
두 개의 이미지가 입력으로 주어질거야. 첫번째 이미지는 판매하고자 하는 제품과 가격정보가 있는 이미지고,
두번째 이미지는 해당 제품의 스펙이야. 이를 바탕으로 마케팅에 사용할 문구를 한국어로 제작해줘.
"""

prompt2 = f"""
두 개의 이미지가 입력으로 주어질거야. 첫번째 이미지는 판매하고자 하는 제품과 가격정보가 있는 이미지고,
두번째 이미지는 해당 제품의 스펙이야. 이를 바탕으로 마케팅에 사용할 문구를 한국어로 제작해줘.
문구는 50단어 이내로 작성해줘.
"""

img1 = Image.open('images/chair.jpg')
img2 = Image.open('images/chair_spec.jpg')
response = img_TextGeneration(prompt2, img1, img2)
print(response)

최고의 스타일과 편안함을 자랑하는 이 의자! 지금 $603.88로 50% 할인된 가격에 만나보세요. 우드 프레임과 고급 비닐 소재, 그리고 Greenguard Gold 인증을 받은 제품입니다. 최대 400 lbs까지 견디는 튼튼함과 5년 보증까지! 다양한 색상 선택 가능, 서두르세요.


In [5]:
prompt3 = f"""
두 개의 이미지가 입력으로 주어질거야. 첫번째 이미지는 판매하고자 하는 제품과 가격정보가 있는 이미지고,
두번째 이미지는 해당 제품의 스펙이야. 이를 바탕으로 마케팅에 사용할 문구를 한국어로 제작해줘.
문구는 세 문장 이내로 간결하게 작성 작성해줘.

그 뒤에는 제품 스펙을 요약한 '스펙정보' 표를 보여줘. 출력은 다음 형식으로 제공해줘:

마케팅 문구 뒤에 해당 제품의 스펙을 보여주는 '스펙정보'라는 제목의 표를 만들어줘.

웹페이지에 바로 사용 가능한 HTML의 형태로 만들고 <div> element 안에 정보를 포함시켜 줘.
또한 무게 단위는 kg로, 길이 단위는 cm으로 변환해서 표기해줘.

마크다운 포멧은 포함하지 말아줘.
"""

img1 = Image.open('images/chair.jpg')
img2 = Image.open('images/chair_spec.jpg')
response = img_TextGeneration(prompt3, img1, img2)
#print(response)

from IPython.display import display, HTML
display(HTML(response))

항목,상세 정보
제품 무게,23.59 kg
팔걸이 높이,66.04 cm
등받이 크기,53.34 W x 41.91 H cm
바퀴,4개 고정 바퀴
프레임 소재,원목
내부 팔걸이 높이,24.13 cm
전체 깊이,73.66 cm
전체 높이,84.46 cm
전체 폭,83.18 cm
좌석 높이,45.72 cm
